In [2]:
import sys
from pathlib import Path

# 将项目根目录添加到 Python 路径
# notebook 位于 src/llms/ 目录，项目根目录需要向上两级
project_root = Path.cwd()
# 如果当前目录不是项目根目录，尝试向上查找
if not (project_root / "src").exists():
    project_root = project_root.parent
if not (project_root / "src").exists():
    project_root = project_root.parent

if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from langchain_core.messages import HumanMessage, SystemMessage
from src.llms.fz import fz_k2_chat_model
from src.tools import search_web

In [18]:
# fz_res = fz_k2_chat_model.bind_tools([search_web]).invoke([
#   SystemMessage(content="你是一个AI助手，你可以使用工具搜索网络信息"),
#   HumanMessage(content="搜索一下Python的最新发展")
# ])

# print(f"非流式响应 content_blocks: {fz_res.content_blocks}\n")
# # [{'type': 'text', 'text': '我来帮您搜索一下Python的最新发展情况。'}, {'type': 'tool_call', 'name': 'search_web', 'args': {'query': 'Python最新发展 Python 3.13 新特性 2024'}, 'id': 'search_web:0'}]
# print(f"非流式响应 content: {fz_res.content}\n")
# # 我来帮您搜索一下Python的最新发展情况。


# 最简单的方式：直接访问 tool_calls
fz_res = fz_k2_chat_model.bind_tools([search_web]).stream([
    SystemMessage(content="你是一个AI助手，你可以使用工具搜索网络信息"),
    HumanMessage(content="搜索一下Python的最新发展")
])


for i,chunk in enumerate(fz_res):
    print(f"block {i}: {chunk.content_blocks}")
    for block in chunk.content_blocks:
        if block["type"] == "reasoning":
            print(f"Model reasoning: {block['reasoning']}")
        elif block["type"] == "text":
            print(f"Response: {block['text']}")
        elif block["type"] == "tool_call_chunk":
            print(f"Tool call: {block['name']}({block['args']})")
    # print(chunk.tool_call_chunks)
    # 打印文本内容
    # if chunk.content:
    #     print(chunk.content, end="", flush=True)
    # tool_calls = getattr(chunk.message, 'tool_calls', None) or getattr(chunk, 'tool_calls', None)
    # if tool_calls:
    #     print(f"\n\n[工具调用]: {tool_calls}")

block 0: [{'type': 'text', 'text': '我来'}]
Response: 我来
block 1: [{'type': 'text', 'text': '为您'}]
Response: 为您
block 2: [{'type': 'text', 'text': '搜索'}]
Response: 搜索
block 3: [{'type': 'text', 'text': '一下'}]
Response: 一下
block 4: [{'type': 'text', 'text': 'Python'}]
Response: Python
block 5: [{'type': 'text', 'text': '的最新'}]
Response: 的最新
block 6: [{'type': 'text', 'text': '发展'}]
Response: 发展
block 7: [{'type': 'text', 'text': '情况'}]
Response: 情况
block 8: [{'type': 'text', 'text': '。'}]
Response: 。
block 9: [{'type': 'tool_call_chunk', 'id': 'search_web:0', 'name': 'search_web', 'args': '', 'index': 0}]
Tool call: search_web()
block 10: [{'type': 'tool_call_chunk', 'id': None, 'name': None, 'args': '{"', 'index': 0}]
Tool call: None({")
block 11: [{'type': 'tool_call_chunk', 'id': None, 'name': None, 'args': 'query', 'index': 0}]
Tool call: None(query)
block 12: [{'type': 'tool_call_chunk', 'id': None, 'name': None, 'args': '":', 'index': 0}]
Tool call: None(":)
block 13: [{'type': 'too

In [ ]:

# 测试非流式调用
# fornax_res = FornaxChatModel().invoke([
#     SystemMessage(content="你是一个AI助手"),
#     HumanMessage(content="用一句话介绍你自己")
# ])
# print(f"非流式响应: {fornax_res.content}\n")

# 测试流式调用
# fornax_stream = FornaxChatModel().stream([
#     SystemMessage(content="你是一个AI助手"),
#     HumanMessage(content="你是什么模型")
# ])

# print("流式响应: ", end="", flush=True)
# for chunk in fornax_stream:
#     print(chunk.content, end="", flush=True)